In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [2]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 6.06 s, sys: 5.12 s, total: 11.2 s
Wall time: 2min 40s


In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 14.4 s, sys: 854 ms, total: 15.2 s
Wall time: 15.6 s


In [6]:
%ls figsharerainfall/

ACCESS-CM2_daily_rainfall_NSW.csv
ACCESS-ESM1-5_daily_rainfall_NSW.csv
AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
BCC-CSM2-MR_daily_rainfall_NSW.csv
BCC-ESM1_daily_rainfall_NSW.csv
CMCC-CM2-HR4_daily_rainfall_NSW.csv
CMCC-CM2-SR5_daily_rainfall_NSW.csv
CMCC-ESM2_daily_rainfall_NSW.csv
CanESM5_daily_rainfall_NSW.csv
EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
FGOALS-f3-L_daily_rainfall_NSW.csv
FGOALS-g3_daily_rainfall_NSW.csv
GFDL-CM4_daily_rainfall_NSW.csv
GFDL-ESM4_daily_rainfall_NSW.csv
INM-CM4-8_daily_rainfall_NSW.csv
INM-CM5-0_daily_rainfall_NSW.csv
KIOST-ESM_daily_rainfall_NSW.csv
MIROC6_daily_rainfall_NSW.csv
MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
MPI-ESM1-2-HR_daily_rainfall_NSW.csv
MPI-ESM1-2-LR_daily_rainfall_NSW.csv
MRI-ESM2-0_daily_rainfall_NSW.csv
NESM3_daily_rainfall_NSW.csv
NorESM2-LM_daily_rainfall_NSW.csv
NorESM2-MM_daily_rainfall_NSW.csv
SAM0-UNICON_daily_rainfall_NSW.csv
TaiESM1_daily_rainfall_NSW.csv
__MACOSX/
combined_data.csv
data.zip
observed_daily_rainfall_SYD.csv


In [7]:
pd.read_csv("figsharerainfall/ACCESS-CM2_daily_rainfall_NSW.csv")

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.00,140.625,142.50,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.00,140.625,142.50,1.047658e-02
...,...,...,...,...,...,...
1932835,2014-12-27 12:00:00,-30.00,-28.75,151.875,153.75,2.951144e-02
1932836,2014-12-28 12:00:00,-30.00,-28.75,151.875,153.75,2.257118e-01
1932837,2014-12-29 12:00:00,-30.00,-28.75,151.875,153.75,1.204670e-01
1932838,2014-12-30 12:00:00,-30.00,-28.75,151.875,153.75,2.632404e-02


In [8]:
# %%time
# %%memit
# # Join all data together
# files = glob.glob('figshare/*.csv')
# df = pd.concat((pd.read_csv(file, index_col=0, parse_dates=True)
#                   .assign(model=re.findall("/([^_]*)", file)[0])
#                 for file in files)
#               )

In [10]:
%load_ext memory_profiler

In [11]:
%%time
%%memit

# use_cols = ["ArrDelay", "DepDelay", "Distance", "TailNum","UniqueCarrier","Origin","Dest"]
files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, parse_dates=True)
                .assign(model=re.findall(r'/([^_]*)', file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

peak memory: 5089.48 MiB, increment: 4625.88 MiB
CPU times: user 13min 42s, sys: 25.6 s, total: 14min 8s
Wall time: 14min 21s
